In [ ]:
#ライブラリのインストール

!pip uninstall tensorflow -y

!pip install pydub #オーディオ再生ライブラリ
!pip install spleeter-gpu #音楽分離ライブラリ
!pip install youtube-dl #YouTubeダウンロードライブラリ
!pip install pretty_midi #MIDI書き出し用ライブラリ

In [ ]:
#モジュールの読み込み

import warnings
warnings.simplefilter('ignore')
 
from IPython.display import Audio,display,clear_output,update_display
from google.colab import files
 
import glob
import os
import subprocess
import re
import ipywidgets as widgets
import shutil
 
from pydub import AudioSegment 
from spleeter.separator import Separator
 
MP3_BITRATE = "192k"
 
# file operation
def get_uploaded_music_path_list():
  upload_music_path_list = [simplify_filename(path) for path in glob.glob('/content/*') 
                            if re.search(r".(wav|mp3|ogg|m4a|wma|flac)", path.lower())]
  if(upload_music_path_list):
    return upload_music_path_list
  else:
    raise ValueError("音源が見つかりません．\n拡張子を確認して音源をアップロードしてください．")
 
def get_uploaded_music_name_list():
  return [os.path.basename(p) for p in get_uploaded_music_path_list()]
 
def get_output_folder_path_list():
  output_list_folder_path_list = glob.glob("/content/audio_output/*")
  if(output_list_folder_path_list):
    return output_list_folder_path_list
  else:
    raise ValueError("分割された音源が見つかりません．\n音源を分割してください．")
 
def get_music_name(music_path):
  return os.path.basename(music_path).split(".")[0]
 
def simplify_filename(path):
  file_name = get_music_name(path)
  ext = os.path.basename(path).split(".")[-1]
  renamed_file = re.sub(r"[-()\"#/@;:<>{}`+=~|!?,　 ]", "", file_name)[:20] + "." +ext
  new_file_path = os.path.dirname(path) + "/" + renamed_file
  os.rename(path,new_file_path)
  return new_file_path
 
# spleeter operation
def run_separation(target_music,part_mode):
  music_name = get_music_name(target_music)
  tmp_folder = "/content/audio_output/tmp/"
  output_folder ="/content/audio_output/{name}/".format(name=music_name)
  output_part_folder ="/content/audio_output/{name}/{mode}part/".format(name=music_name,mode=part_mode)
 
  separator = Separator('spleeter:{mode}stems'.format(mode=part_mode))
  separator.separate_to_file(target_music, tmp_folder)
  separated_folder = tmp_folder + music_name + "/"
 
  if(os.path.exists(output_part_folder)):
    shutil.rmtree(output_part_folder)
  os.makedirs(output_folder, exist_ok=True)
  shutil.move(separated_folder,output_part_folder)
 
  # convert wav to mp3
  os.makedirs(output_part_folder[:-1]+"_mp3/",exist_ok=True)
 
  for path in glob.glob(output_part_folder+"*.wav"):
    part_name = os.path.basename(path).split(".")[0]
    AudioSegment.from_wav(path).export(output_part_folder[:-1]+"_mp3/"+part_name+".mp3",format="mp3",bitrate=MP3_BITRATE)
 
# misc operation
def get_part_file_or_make(music_path,part,part_mode,encode):
  music_name = get_music_name(music_path)
  if(encode == "mp3"):
    target_file ="/content/audio_output/{name}/{mode}part_mp3/{part}.mp3".format(name=music_name,mode=part_mode,part=part)
  else:
    target_file ="/content/audio_output/{name}/{mode}part/{part}.wav".format(name=music_name,mode=part_mode,part=part)
 
  if(os.path.exists(target_file)):
    return target_file
  else:
    run_separation(music_path,part_mode)
    return target_file

In [ ]:
#URLから動画データのダウンロード

Youtube_url = input()
import youtube_dl
ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl':  "%(title)s" + '.%(ext)s',
    'postprocessors': [
        {'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
         'preferredquality': '192'},
        {'key': 'FFmpegMetadata'},
    ],
}
ydl = youtube_dl.YoutubeDL(ydl_opts)
info_dict = ydl.extract_info(Youtube_url, download=True)

In [ ]:
#音楽データの分離

class try_listen:
  def __init__(self):
    self.select_mode = widgets.Dropdown(
    options=[('5パート', 5), ('4パート', 4), ('2パート', 2)],
    description='分割モード:',
    )
    self.select_music = widgets.Dropdown(
      options=get_uploaded_music_name_list(),
      description='視聴する曲:',
    )
    self.listen_button = widgets.Button(
    description='視聴',
    tooltip='曲を選択して，視聴ボタンを押してください',
    icon='check'
    )
 
    self.listen_music_selecter = widgets.AppLayout(
      header=None,
      left_sidebar=self.select_music,
      center=self.select_mode,
      right_sidebar=self.listen_button,
      footer=None               
    )
    
    self.output = widgets.Output(layout={'border': '1px solid black'})
    self.listen_button.on_click(self.listen_button_clicked)
    display(self.listen_music_selecter)
    display(self.output)
 
  def set_button_disabled(self,mode):
    self.listen_button.disabled=mode
    self.select_music.disabled=mode
    self.select_mode.disabled=mode
 
  def listen_button_clicked(self,b):
    PART5 = ["vocals","piano","other","drums","bass"]
    PART4 = ["vocals","others","drums","bass"]
    PART2 = ["vocals","accompaniment"]
 
    self.set_button_disabled(True)
    
    target_path = "/content/"+self.select_music.value
 
    if(self.select_mode.value == 5):
      parts = PART5
    elif(self.select_mode.value == 4):
      parts = PART4
    elif(self.select_mode.value == 2):
      parts = PART2
 
    for p in parts:
      destination_file = get_part_file_or_make(
          target_path,p,self.select_mode.value,"mp3"
      )
      print(target_path,p,self.select_mode.value,"mp3")
      print(self.select_music.value+": "+p)

      display(Audio(destination_file))
    
    self.set_button_disabled(False)
 
a = try_listen()

In [ ]:
#分離データの解析、MIDI化

import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import pretty_midi


class music_upload:
  def __init__(self):
    self.select_music = widgets.Dropdown(
      options=get_output_folder_path_list(),
      description='アップロードする曲:',
    )
    self.select_output = widgets.Dropdown(
        options=[('ボーカル', 1), ('ベース', 2), ('ピアノ', 3), ('ドラム', 4), ('その他', 5)],
        description='MIDI化するファイル',
    )
    self.upload_button = widgets.Button(
        description='アップロード',
        tooltip='Librosaにアップロードします',
        icon='check'
    )
    self.upload_music_selecter = widgets.AppLayout(
          header=None,
          left_sidebar=self.select_music,
          center=self.select_output,
          right_sidebar=self.upload_button,
          footer=None               
    )

    self.output2 = widgets.Output(layout={'border': '1px solid black'})
    self.upload_button.on_click(self.upload_button_clicked)
    display(self.upload_music_selecter)
    display(self.output2)

  def set_button_disabled(self,mode):
    self.upload_button.disabled=mode
    self.select_music.disabled=mode
    self.select_output.disabled=mode
#アップロードボタンを押したときの処理
  def upload_button_clicked(self,b):
    self.set_button_disabled(True)
    #partの選択
    if(self.select_output.value == 1):
      parts = "vocals"
    elif(self.select_output.value == 2):
      parts = "bass"
    elif(self.select_output.value == 3):
      parts = "piano"
    elif(self.select_output.value == 4):
      parts = "drums"
    elif(self.select_output.value == 5):
      parts = "other"

    #librosaにアップロード

    drive.mount('/content/drive')

    target_path = "{name}/5part_mp3/{part}.mp3".format(name=self.select_music.value,part=parts)

    filename = (target_path)

    y, sr = librosa.load(filename, sr=44100)
      
    self.set_button_disabled(False)

    print(target_path)

    print("MIDI化します")

    #midi化の処理

    y, index = librosa.effects.trim(y)

    onset_frames = librosa.onset.onset_detect(y=y, sr=sr)
    onset_times = librosa.frames_to_time(onset_frames, sr=sr)

    hop_length = 512
    window = 'hann'
    n_chroma = 12
    bins_per_octave = 12
    n_octaves = 7
    n_bins = bins_per_octave * n_octaves
    chroma_cq = librosa.feature.chroma_cqt(y=y, sr=sr, hop_length=hop_length, fmin=librosa.note_to_hz('C1'), n_chroma=n_chroma, n_octaves=n_octaves)
    cqt_amplitude = np.abs(librosa.cqt(y, sr=sr, hop_length=hop_length, fmin=librosa.note_to_hz('C1'), n_bins=n_bins, bins_per_octave=bins_per_octave, window=window))
    end_frames = onset_frames[1:]
    end_frames = np.append(end_frames, chroma_cq.shape[1])

    # Pretty MIDIオブジェクトを作る。
    cello_c_chord = pretty_midi.PrettyMIDI()

    # Instrument Instanceを作る。

    inst = pretty_midi.program_to_instrument_name(1)

    #対応するGeneral MIDI program numberを返してくれる
    cello_program = pretty_midi.instrument_name_to_program(inst)

    # Instrument instanceを作成
    cello = pretty_midi.Instrument(program=cello_program)

    for onset_frame, end_frame in zip(onset_frames, end_frames):

      onset_time = librosa.core.frames_to_time(onset_frame, sr=sr, hop_length=hop_length)
      end_time = librosa.core.frames_to_time(end_frame, sr=sr, hop_length=hop_length)

      chroma_cq_mean = np.mean(cqt_amplitude[:, onset_frame:end_frame], axis=1)
      chord_indices = np.argsort(chroma_cq_mean)[-1:-3:-1]
      notes = librosa.midi_to_note(chord_indices)

      print(f't: {onset_time:.2f}-{end_time:.2f}[s], notes: {notes}', end='')
      print()

      for note_name in chord_indices:
            # NoteNameからNote Numberを検索しています。
            subnote_number = pretty_midi.note_number_to_name(note_name)

            note_number = pretty_midi.note_name_to_number(subnote_number)

            # NoteInstanceを作成します。音(pitch)の開始時間と終了時間、
            # velocityを定義します。
            note = pretty_midi.Note(
                velocity=100, pitch=note_number, start=onset_time, end=end_time)

            # 上記で作成したNoteInstanceをCelloInstrumentに加えます。
            cello.notes.append(note)

    onset_envelope = librosa.onset.onset_strength(y, sr=sr)
    times = librosa.times_like(onset_envelope, sr=sr)

    # ChelloInstrumentをPrettyMIDIオブジェクトに加えます。
    cello_c_chord.instruments.append(cello)

    # PrettyMIDIオブジェクトをMIDIファイルとして書き出しましょう。
    cello_c_chord.write('dhhd.mid')

    print("完成しました。")

b = music_upload()

In [ ]:
#Constant-Q plt

fig = plt.figure()
    ax = fig.add_subplot()
    librosa.display.specshow(librosa.amplitude_to_db(cqt_amplitude, ref=np.max), sr=sr, x_axis='time', y_axis='cqt_note')
    plt.colorbar(format='%+2.0f dB')
    ax.set_title('constant-Q power spectrum')
    plt.tight_layout()
    plt.show()

In [ ]:
#onset plt

fig = plt.figure(figsize=(20, 20))

    ax1 = fig.add_subplot(2, 1, 1)
    librosa.display.specshow(chroma_cq, x_axis='time', y_axis='chroma')
    ax1.set_xlabel('Time [s]')
    ax1.set_title('chroma cqt')

    ax2 = fig.add_subplot(2, 1, 2, sharex=ax1)
    ax2.plot(times, onset_envelope, label='onset strength')
    ax2.vlines(onset_times, 0, onset_envelope.max(), color='r', alpha=0.9, linestyle='--', label='onsets')
    ax2.set_xlabel('Time [s]')
    ax2.legend(frameon=True, framealpha=0.75)
    ax2.set_title('onsets')

In [ ]:
#音階の解析結果

print(f't: {onset_time:.2f}-{end_time:.2f}[s], notes: {notes}', end='')
print()